In [1]:
import pandas as pd
import numpy as np

dist = pd.read_csv("data/processed/distribution_mvp_etherscan.csv")
dist.head()


,coingecko_id,token_name,symbol,contract_address,tier,category,chain,verified_contract,verify_note,tx_sampled,unique_senders,unique_receivers,unique_addresses,top10_sender_share,top10_receiver_share,note
0,uniswap,Uniswap,UNI,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,A,DEX,ethereum,True,NaN,5000,854,1077,1302,0.4290,0.4324,NaN
1,chainlink,Chainlink,LINK,0x514910771af9ca656af840dff83e8264ecf986ca,A,Infrastructure & Middleware,ethereum,True,NaN,5000,1385,1582,1994,0.2214,0.2182,NaN
2,aave,Aave,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,A,Lending & Borrowing,ethereum,True,NaN,5000,838,932,1129,0.3118,0.3332,NaN
3,maker,Maker,MKR,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,A,Lending & Borrowing,ethereum,True,NaN,5000,1470,332,1554,0.3444,0.6560,NaN
4,lido-dao,Lido Dao,LDO,0x5a98fcbea516cf06857215779fd812ca3bef1b32,A,Liquid staking,ethereum,True,NaN,5000,744,904,1075,0.5074,0.5104,NaN


In [2]:
dist["peer_group"] = dist["tier"].astype(str) + "_" + dist["category"].astype(str)


In [3]:
def percentile_score(series, higher_is_better=True):
    pct = series.rank(pct=True)
    if not higher_is_better:
        pct = 1 - pct
    return (pct * 100).clip(0, 100)


In [4]:
dist["breadth_score"] = dist.groupby("peer_group")["unique_addresses"].transform(
    lambda x: percentile_score(x, higher_is_better=True)
)

dist["sender_conc_score"] = dist.groupby("peer_group")["top10_sender_share"].transform(
    lambda x: percentile_score(x, higher_is_better=False)
)

dist["receiver_conc_score"] = dist.groupby("peer_group")["top10_receiver_share"].transform(
    lambda x: percentile_score(x, higher_is_better=False)
)

dist["verification_score"] = np.where(dist["verified_contract"], 100, 0)


In [5]:
dist["distribution_health_score"] = (
    0.40 * dist["breadth_score"] +
    0.20 * dist["sender_conc_score"] +
    0.25 * dist["receiver_conc_score"] +
    0.15 * dist["verification_score"]
).clip(0, 100)


In [6]:
dist.to_csv("data/processed/distribution_health_scores.csv", index=False)

dist[["token_name","symbol","distribution_health_score"]].sort_values(
    "distribution_health_score", ascending=False
)


,token_name,symbol,distribution_health_score
19,Voyager Token,VGX,77.5
16,Shiba Inu,SHIB,77.5
5,Curve DAO,CRV,77.5
2,Aave,AAVE,57.5
1,Chainlink,LINK,55.0
17,FTX Token,FTT,55.0
14,Basic Attention Token,BAT,55.0
13,Yearn Finance,YFI,55.0
12,Sushi,SUSHI,55.0
11,Rocket Pool,RPL,55.0
